# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()#посмотрим информацию о нашем датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что у 21525 строк, в столбцах `days_employed`и `total_income` имеются пропуски.

In [3]:
data.head(40)#посмотрим первые 40 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

Мы видим, что в столбцах `days_employed` и `total_income` встречаются пропуски.

In [4]:
print('Процент пропусков для столбца days_employed: {:.0%}'.format(data['days_employed'].isnull().sum()/data.shape[0])) 
print('Процент пропусков для столбца total_income: {:.0%}'.format(data['total_income'].isnull().sum()/data.shape[0])) 

Процент пропусков для столбца days_employed: 10%
Процент пропусков для столбца total_income: 10%


Видим, что проценты пропусков совпадают, проверим, возсможно это строки где пропуски в обоих столбцах сразу

In [5]:
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Действительно, так оно и есть, пропуски попадаются сразу в двух столбацх. Это явно не связано с тем, что это люди не работают, т.к. столбец `income_type` говорит о другом.  Пропуски возникли в связи с человечески фактором, либо при переносе данных, возможно из-за неверной типизации. Переменные в обоих столбцах количественные, поэтому заменим их медианной, но не среднем, так как есть значения сильно выделящиеся среди большинства. Столбец `days_employed` сначала избавим от аномальных значений.

In [6]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [7]:
data.head(50)#посмотрим первые 50 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Начнем изучать столбцы на предмет аномальных значений и избавляться от них.

In [8]:
display(data['gender'].value_counts())#посмотрим аномальные значения в данном столбце
data = data[data['gender']!= 'XNA']

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Обратим внимание что в столбце `days_employed` есть отрицательные значения, а у пенсионеров числа будто записанные в часах, а не днях. Скорее всего эти аномалии вызваны человеческим фактором.

In [9]:
data.loc[data['income_type']=='пенсионер','days_employed'] = data.loc[data['income_type']=='пенсионер','days_employed']/24
#переведем часы в дни

In [10]:
data.loc[data['days_employed']<0,'days_employed'] = data['days_employed']*(-1)
#избавимся от отрицательных значений

In [11]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
#теперь заполним пропуски медианным значением

In [12]:
display(data['children'].value_counts())
data.loc[data['children']== -1,'children'] = 1 #избавимся от аномальных значений
data.loc[data['children']== 20,'children'] = 2

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [13]:
display(data['dob_years'].value_counts())
data.loc[data['dob_years'] == 0,'dob_years'] = data['dob_years'].median()#заменим аномальные значения медианой

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    263
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Мы избавились от всех аномальных значений в нашем датасете.

### Шаг 2.3. Изменение типов данных.

In [14]:
data['total_income'] = data['total_income'].astype('int')#приведем значения в обоих столбцах к int
data['dob_years'] = data['dob_years'].astype('int')

### Шаг 2.4. Удаление дубликатов.

In [15]:
data = data.drop_duplicates().reset_index(drop=True)#избавимся от явных дубликатов
data['education'] = data['education'].str.lower()#избавимся от дубликатов вызванных разным форматированием

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для удобства составим "словари" для нашего датафрейма, которые будут состоять из категорий и индексов к ним.

In [16]:
data_education = data[['education_id','education']]
data_education = data_education.drop_duplicates().reset_index(drop=True)#создаем словарь с удалением 
data_education = data_education.sort_values('education_id')             #дубликатов и новыми индексами

In [17]:
data_family = data[['family_status_id','family_status']]
data_family = data_family.drop_duplicates().reset_index(drop=True)#по аналогии, удаляем дубликаты и делаем новые индексы
data_family.sort_values('family_status_id')

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [18]:
data = data.drop(columns=['education','family_status'],axis=1)#избавляемся от категорий и оставляем только их индексы


### Шаг 2.6. Категоризация дохода.

In [19]:
def income_type (row): #создаем функцию, которая будет присваивать категорию в зависимости от величины дохода
    income = row['total_income']
    if income <= 30000:
        return 'E'
    if (income>=30001) & (income<=50000):
        return 'D'
    if (income>=50001) & (income<=200000):
        return 'C'
    if (income>=200001) & (income<=1000000):
        return 'B'
    return 'A'
data['total_income_category'] = data.apply(income_type,axis=1)#применяем функцию ко всему датасету

### Шаг 2.7. Категоризация целей кредита.

In [20]:
def purpose_categorizer(row):#создадим функцию, которая будет присваивать категорию в зависимости причины
    purpose = row['purpose']
    if 'свадьб' in purpose:
        return 'проведение свадьбы'    
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    return 'операции с недвижимостью'
data['purpose_category'] = data.apply(purpose_categorizer,axis=1)#применяем функцию  ко всему датасету
data.head(40)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

Ответим на вопрос: есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
data_children_pivot = data.pivot_table(index=['children'], values='debt', aggfunc='mean')
display(data_children_pivot) #построим pivot_table для наглядности
print('Нет детей: {:.0%}'.format(data_children_pivot.loc[0,'debt'])) #посчитаем для каждого столбца отношение 
print('Для 1 ребенка: {:.0%}'.format(data_children_pivot.loc[1,'debt']))#людей с задолженностями к людям без задолженностей
print('Для 2 детей: {:.0%}'.format(data_children_pivot.loc[2,'debt']))
print('Для 3 детей: {:.0%}'.format(data_children_pivot.loc[3,'debt']))
print('Для 4 детей: {:.0%}'.format(data_children_pivot.loc[4,'debt']))

,debt
children,
0,0.075363
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


Нет детей: 8%
Для 1 ребенка: 9%
Для 2 детей: 9%
Для 3 детей: 8%
Для 4 детей: 10%


Видим, что с увеличением количества детей наблюдается тенденция появлениям задолженностей по кредитам.

##### Вывод 1:

Ответим на вопрос: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
data_family_category = data.merge(data_family,on='family_status_id',how='left')
data_family_pivot = data_family_category.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
display(data_family_pivot)
print('Не женат/ не замужем: {:.0%}'.format(data_family_pivot.loc['Не женат / не замужем','debt'])) 
print('в разводе: {:.0%}'.format(data_family_pivot.loc['в разводе','debt']))
print('вдовец/ вдова: {:.0%}'.format(data_family_pivot.loc['вдовец / вдова','debt']))
print('гражданский брак: {:.0%}'.format(data_family_pivot.loc['гражданский брак','debt']))
print('женат/ замужем: {:.0%}'.format(data_family_pivot.loc['женат / замужем','debt']))

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093247
женат / замужем,0.075421


Не женат/ не замужем: 10%
в разводе: 7%
вдовец/ вдова: 7%
гражданский брак: 9%
женат/ замужем: 8%


Видим, что категории *в разводе* и *вдовец/ вдова* имеют самые низкие проценты, в то время как остальные категории показывают рост.

##### Вывод 2:

Ответим на вопрос: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
data_income_pivot = data.pivot_table(index=['total_income_category'], values='debt', aggfunc='mean')
display(data_income_pivot)
print('Категория A: {:.0%}'.format(data_income_pivot.loc['A']['debt'])) 
print('Категория B: {:.0%}'.format(data_income_pivot.loc['B']['debt']))
print('Категория C: {:.0%}'.format(data_income_pivot.loc['C']['debt']))
print('Категория D: {:.0%}'.format(data_income_pivot.loc['D']['debt']))
print('Категория E: {:.0%}'.format(data_income_pivot.loc['E']['debt']))

,debt
total_income_category,
A,0.080000
B,0.070635
C,0.084830
D,0.060000
E,0.090909


Категория A: 8%
Категория B: 7%
Категория C: 8%
Категория D: 6%
Категория E: 9%


Видим, что с увеличением дохода шансы возврата долга вовремя растут, хотя явная зависимость от этого параметра отсутствует.

##### Вывод 3:

Ответим на вопрос: Как разные цели кредита влияют на его возврат в срок?

In [24]:
data_purpose_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')
display(data_purpose_pivot)
print('операции с автомобилем: {:.0%}'.format(data_purpose_pivot.loc['операции с автомобилем']['debt'])) 
print('операции с недвижимостью: {:.0%}'.format(data_purpose_pivot.loc['операции с недвижимостью']['debt']))
print('получение образования: {:.0%}'.format(data_purpose_pivot.loc['получение образования']['debt']))
print('проведение свадьбы: {:.0%}'.format(data_purpose_pivot.loc['проведение свадьбы']['debt']))

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072320
получение образования,0.092177
проведение свадьбы,0.079692


операции с автомобилем: 9%
операции с недвижимостью: 7%
получение образования: 9%
проведение свадьбы: 8%


Видим, что наиболее благоприятными категориями являются *проведение свадьбы* и *операции с недвижимостью*, в то время как 2 другие показывают небольшой рост.

## Общий вывод:

Мы обработали данные, избавились от пропусков и аномальных значений, создали словари и присвоили категории по *причине выдачи кредита* и *общего дохода*. Выбрали критерии для дальнейшего использования в анализе, с целью более точно предсказать будет ли выплачен кредит в срок.
Согласное нашему исследованию идеальный клиент выглядит так: клиент **в разводе** или **вдовец/вдова**, у клиента **нет детей**,**доход 30001–50000**, берет кредит с целью **операции с недвижимостью**.
Клиент с наибольшей вероятностью просрочки выглядит так: у клиента **4 детей**, клиент **не женат/не замужем**, доход в пределах **0-30000**, кредит берется с целью **получение образования** или **операции с автомобилем**